LIMPIAR AMBOS CSV

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import DATA_PATH, DATA_PROCESADA_PATH, PATRON_MOSTRADOR_PATH, MOSTRADOR_SALIDA_PATH, PATRON_FACTURACION_PATH, FACTURACION_SALIDA_PATH

LIMPIO MOSTRADOR

In [ ]:

# 1. Leer archivo consolidado
df = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')

# 2. Eliminar ventas anuladas (si existe la columna "Anulado")
if "Anulado" in df.columns:
    df = df[df["Anulado"].str.lower() == "no"]
else:
    print("No se encontró columna 'Anulado'.")

# 3. Estandarizar columna de fecha
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# 4. Asegurar que TotalVenta y Cantidad sean numéricos
df['TotalVenta'] = pd.to_numeric(df['TotalVenta'].round(0).astype('Int64') , errors='coerce')
if 'Cantidad' in df.columns:
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')


columnas_utiles = ['Fecha', 'Producto', 'Cantidad', 'TotalVenta','Cond.Pago','Cliente']
df = df[columnas_utiles]

# 6. Guardar CSV limpio
df.to_csv(MOSTRADOR_SALIDA_PATH, index=False, encoding='utf-8-sig')
print("Archivo limpio guardado en data_procesada/ventas_limpio.csv")

# Verificá primeras filas
df.head()


ELIMINO EL EFECTO INFLACIONARIO


In [ ]:
# 1. Leé tus ventas
df = pd.read_csv(MOSTRADOR_SALIDA_PATH, parse_dates=['Fecha'])

# 2. Filtrá solo enero-junio de 2024 y 2025
df['Anio'] = df['Fecha'].dt.year
df['Mes'] = df['Fecha'].dt.month
df = df[((df['Anio'] == 2024) | (df['Anio'] == 2025)) & (df['Mes'] >= 1) & (df['Mes'] <= 6)]

# 3. Armá la tabla de inflación acumulada 

inflacion = pd.DataFrame({
    'Anio': [2024]*6 + [2025]*6,
    'Mes':  [1,2,3,4,5,6,1,2,3,4,5,6],
    'Coef_Ajuste': [2.426,2.013,1.779,1.603,1.474,1.415,1.227,1.202,1.173,1.134,1.015,1.000]
})


# 4. Mergea los coeficientes al dataset de ventas
df = df.merge(inflacion, on=['Anio','Mes'], how='left')

# 5. Ajustá el monto de cada venta a valores de junio 2025
df['TotalVenta_Ajustado'] = df['TotalVenta'] * df['Coef_Ajuste']

# Redondeá y convertí a entero
df['TotalVenta_Ajustado'] = df['TotalVenta_Ajustado'].round(0).astype('Int64')

# 6. Guardá el dataset ajustado
df.to_csv(MOSTRADOR_SALIDA_PATH, index=False, encoding="utf-8-sig")
print("¡Dataset ajustado guardado como ventas_mostrador.csv!")


LIMPIO FACTURACION

In [7]:
# 1. Leé tus ventas
df = pd.read_csv(FACTURACION_SALIDA_PATH, parse_dates=['fecha_emision'])

# 2. Filtrá solo enero-junio de 2024 y 2025
df['Anio'] = df['fecha_emision'].dt.year
df['Mes'] = df['fecha_emision'].dt.month
df = df[((df['Anio'] == 2024) | (df['Anio'] == 2025)) & (df['Mes'] >= 1) & (df['Mes'] <= 6)]

# 3. Armá la tabla de inflación acumulada 

inflacion = pd.DataFrame({
    'Anio': [2024]*6 + [2025]*6,
    'Mes':  [1,2,3,4,5,6,1,2,3,4,5,6],
    'Coef_Ajuste': [2.426,2.013,1.779,1.603,1.474,1.415,1.227,1.202,1.173,1.134,1.015,1.000]
})


# 4. Mergea los coeficientes al dataset de ventas
df = df.merge(inflacion, on=['Anio','Mes'], how='left')

# 5. Ajustá el monto de cada venta a valores de junio 2025
df['TotalVenta_Ajustado'] = df['importe_total'] * df['Coef_Ajuste']

# Redondeá y convertí a entero
df['TotalVenta_Ajustado'] = df['TotalVenta_Ajustado'].round(0).astype('Int64')

# 6. Guardá el dataset ajustado
df.to_csv(FACTURACION_SALIDA_PATH, index=False, encoding="utf-8-sig")
print("¡Dataset ajustado guardado como ventas_facturacion!")

¡Dataset ajustado guardado como ventas_facturacion!
